In [1]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

Generating All the tokens from dataset 

In [2]:
def preprocess_data(with_pos_tag = False):
  total_states = []
  f = open("dataset.txt", "r",encoding = 'utf-8')
  lines = f.readlines()
  line = ""
  for word in lines:
    word = word.split('\t')

    if(len(word) != 1):
      tag = word[1].strip()
      word = word[0].strip()
      
      if(with_pos_tag == False):
        total_states.append(word)
      else:
        total_states.append(word + " " + tag)
  
  import random
  random.shuffle(total_states)

  train_size = (len(total_states) * 80)//100

  train_set = total_states[:train_size]
  test_set = total_states[train_size:]

  vectorizer = TfidfVectorizer().fit(total_states)
  X_train = vectorizer.transform(train_set)
  X_test = vectorizer.transform(test_set)
  return train_set, test_set, X_train, X_test

In [3]:
train_set, test_set, X_train, X_test = preprocess_data()

Training KMeans Model for the Train Set

In [4]:
from sklearn.cluster import KMeans
modelkmeans = KMeans(n_clusters=2, init='k-means++', n_init=100, n_jobs = -1)
modelkmeans.fit(X_train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=100, n_jobs=-1, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

Generating Result Set For Measuring Accuracy



In [5]:
Y_train = modelkmeans.predict(X_train)
Y_test = modelkmeans.predict(X_test)

  Calculating the Accuracy of the model

In [6]:
def measure_accuracy(train_set, test_set, Y_train, Y_test, with_pos_tag = True):
  file2 = open('hin-token-chunk-conll-treebank.txt', 'r')
  result_lines = file2.readlines()
  tags = {}

  for line in result_lines:
    word = line.split('\t')
    if(len(word) != 1):
      token = word[0].strip()
      word = word[1].strip()
      if(word[0] == 'B'):
        tags[token] = 0

      else:
        tags[token] = 1

  if(with_pos_tag == True):
    for i in range(len(train_set)):
      train_set[i] = train_set[i].split()[0]

    for i in range(len(test_set)):
      test_set[i] = test_set[i].split()[0]


  total_correct = 0
  for i in range(len(Y_train)):
    if(Y_train[i] == tags[train_set[i]]):
      total_correct += 1

  print("Training Accuracy is : ", max(100 - (total_correct / len(train_set)*100), (total_correct / len(train_set)*100)))

  total_correct = 0
  for i in range(len(Y_test)):
    if(Y_test[i] == tags[test_set[i]]):
      total_correct += 1

  print("Testing Accuracy is : ", max(100 - (total_correct / len(test_set)*100), (total_correct / len(test_set)*100)))

In [7]:
measure_accuracy(train_set, test_set, Y_train, Y_test)

Training Accuracy is :  55.34537817543658
Testing Accuracy is :  55.400584593431375


In [8]:
pickle.dump(modelkmeans, open("model_tfid_without_pos.pkl", "wb"))

Now Training Same model with Pos Tags

In [9]:
train_set, test_set, X_train, X_test = preprocess_data(with_pos_tag = True)

In [10]:
from sklearn.cluster import KMeans
modelkmeans = KMeans(n_clusters=2, init='k-means++', n_init=100, n_jobs = -1)
modelkmeans.fit(X_train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=100, n_jobs=-1, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [11]:
Y_train = modelkmeans.predict(X_train)
Y_test = modelkmeans.predict(X_test)

In [12]:
measure_accuracy(train_set, test_set, Y_train, Y_test, with_pos_tag=True)

Training Accuracy is :  72.68189533645962
Testing Accuracy is :  72.82906396004512


In [13]:
pickle.dump(modelkmeans, open("model_tfid_with_pos.pkl", "wb"))